In [1]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import pytorch_lightning as pl
from pitchclass2vec import encoding, model
from pitchclass2vec.pitchclass2vec import Pitchclass2VecModel
from tasks.segmentation.data import BillboardDataset, SegmentationDataModule
from tasks.segmentation.functional import LSTMBaselineModel

from evaluate import load_pitchclass2vec_model

RANDOM_SEED = 42
pl.seed_everything(seed=RANDOM_SEED)

Global seed set to 42


42

# Segmentation baseline

In [5]:
EXP = [
    #("text", "fasttext", "out/fasttext_best/model.ckpt"),
    ("timed-root-interval", "emb-weighted-fasttext", "out/rootinterval_best/model.ckpt"),
    #("rdf", "randomwalk-rdf2vec", "out/rdf2vec_best/model.ckpt"),
]

  
experiments_df = pd.DataFrame(columns=[
    "encoding", "model", "path", "test_p_precision", "test_p_recall",  "test_p_f1",  "test_under",  "test_over",  "test_under_over_f1"
])

In [6]:
import logging
logging.disable(logging.CRITICAL)


for exp in tqdm(EXP):    
    p2v = load_pitchclass2vec_model(*exp)
    data = SegmentationDataModule(BillboardDataset, p2v, 256)
        
    lstm_model = LSTMBaselineModel(embedding_dim=p2v.vector_size, hidden_size=256, num_layers=5, dropout=0.2, learning_rate=0.001)
    trainer = pl.Trainer(max_epochs=150, accelerator="auto", devices=1,
                         enable_progress_bar=False)
    trainer.fit(lstm_model, data)
    test_metrics = trainer.test(lstm_model, data)
    experiments_df = experiments_df.append({
        "encoding": exp[0], "model": exp[1], "path": exp[2],
        **test_metrics[0]
    }, ignore_index=True)

  0%|          | 0/1 [00:00<?, ?it/s]


 92%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 822/890 [00:01<00:00, 813.56it/s]

Track 974 not parsable


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 890/890 [00:01<00:00, 813.69it/s]
/home/nicolas/pitchclass2vec/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1555: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(

 92%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉           | 823/890 [00:01<00:00, 818.06it/s]

Track 974 not parsable


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 890/890 [00:01<00:00, 814.85it/s]


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.5265165567398071
        test_over           0.9863528000290747
        test_p_f1           0.5928337247292093
    test_p_precision        0.6138726851018923
      test_p_recall         0.6105689214759769
       test_under            0.986965474488972
   test_under_over_f1       0.9866590421476571
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/tmp/ipykernel_393440/3128048165.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  experiments_df = experiments_df.append({


In [7]:
experiments_df

,encoding,model,path,test_p_precision,test_p_recall,test_p_f1,test_under,test_over,test_under_over_f1,test_loss
0,timed-root-interval,emb-weighted-fasttext,out/rootinterval_best/model.ckpt,0.613873,0.610569,0.592834,0.986965,0.986353,0.986659,0.526517
